In [7]:
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
from PIL import Image, ImageEnhance
import math
import os
import pandas as pd
from skimage import io

from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # to tensor before normalize
])
def load_image(file):
    return Image.open(file)
def image_path(root, basename, extension):
    return os.path.join(root, f'{basename}{extension}')
os.listdir('../input/hsgs-hackathon2022/train_data/Train_labels')
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, label_dir, transform=transform):
        video_files = os.listdir(img_dir)
        self.labels = list()
        self.imgs = list()
        for file_name in video_files: 
            cur_csv = pd.read_csv(os.path.join(label_dir, file_name+'.csv'))
            cur_len = len(cur_csv.index)
            for itr in range(cur_len):
                img_file_dir = os.path.join(os.path.join(img_dir, file_name), cur_csv.iloc[itr, 0] + '.PNG')
                self.imgs.append(img_file_dir) # image directory
                self.labels.append([cur_csv.iloc[itr, 0], cur_csv.iloc[itr, 1]]) # label
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform

            
    def __getitem__(self, index):
        with open(self.imgs[index], 'rb') as f:
            image = load_image(f).convert('RGB')
        label = self.labels[index][1]
        filename = self.labels[index][0]
        
#         image, label = self.spm_transform(image, label)
        
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.labels)

In [9]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 64
num_epochs = 10

# Load Data

dataset = CustomImageDataset(
    img_dir='../input/hsgs-hackathon2022/train_data/Train',
    label_dir='../input/hsgs-hackathon2022/train_data/Train_labels'
)
n_train = math.floor(0.9*len(dataset)) 
n_val = len(dataset) - math.floor(0.9*len(dataset)) 
train_dataset, validation_dataset = torch.utils.data.random_split(dataset, [n_train, n_val])
train=train_dataset
val=validation_dataset

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, drop_last = True)
validation_loader = DataLoader(validation_dataset, batch_size = batch_size, shuffle=False, drop_last = True)


cuda


In [10]:

model=torchvision.models.googlenet(pretrained=True)
model=model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# Train Network
from tqdm import tqdm
min_val_loss=100000000000
for epoch in range(num_epochs):
  total_loss_train=0
  total_acc_train=0
  for x, y in tqdm(train_loader):

    x = x.to(device)
    y = y.to(device)
    output = model(x.float())

    batch_loss = criterion(output, y)
    total_loss_train += batch_loss.item()

    acc = (output.argmax(dim=1)==y).sum().item()
    total_acc_train += acc

    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
  
  total_loss_val=0
  total_acc_val=0

  with torch.no_grad():
    for x, y in tqdm(validation_loader):

      x = x.to(device)
      y = y.to(device)
      output = model(x)
      batch_loss = criterion(output, y)
      total_loss_val += batch_loss.item()

      acc = (output.argmax(dim=1)==y).sum().item()
      total_acc_val += acc
  
  print(
      f'Epochs: {epoch+1} | Train Loss: {total_loss_train / len(train):.3f}\
      | Train Accuracy: {total_acc_train/len(train):.3f}\
      | Val Loss: {total_loss_val/len(val):.3f}\
      | Val Accuracy:{total_acc_val/len(val):.3f}'
  )

  if min_val_loss>total_loss_val/len(val):
    min_val_loss = total_loss_val/len(val)
    torch.save(model.state_dict(), "ModelA.pt")
    print(f"Save model because val loss improve loss {min_val_loss:.3f}")

  0%|          | 0/357 [00:01<?, ?it/s]


KeyboardInterrupt: 